### In this notebook, I first generate faces using captions from COCO dataset using ERNIE-ViLG, and then, I using face detector to extract faces from the images generated. 

In [1]:
#!pip install translators --upgrade

Load caption data

In [1]:
import paddlehub as hub
import translators as ts
lines = open('face_captions.txt').read().splitlines()

/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Using state Virginia server backend.


In [2]:
import random
random.seed(42)
#Generate 1000 random numbers between 0 and 9999
# I plan to generate about 600 images, but may need to come back generate more if some of the images do not contain a good face
randomlist = random.sample(range(0, 9999), 1000)

In [3]:
len(randomlist)

1000

In [4]:
randomlist[:10]

[1824, 409, 4506, 4012, 3657, 2286, 1679, 8935, 1424, 9674]

Start generating face images

In [6]:
import paddlehub as hub
import translators as ts
module = hub.Module(name="ernie_vilg")
for i in range(181,350):
    prompt = ts.baidu(lines[randomlist[i]], professional_field='common')
    try: # for certain prompts, error message "暂不支持创作该内容，请修改后再试 (does not support this prompt, please modify)" was returned
        module.generate_image(text_prompts=prompt, style='写实风格',  output_dir='./generated_face_images/',topk=1)
        print(i)      
    except:
        continue

  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
100


  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
101


  0%|          | 0/100 [00:00<?, ?%/s]

...

Saving Images...
Done
177


  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
178
暂不支持创作该内容，请修改后再试


  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
180
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限
日访问量超限


extract faces from the generated face images 

In [ ]:
#set up folders for extracted faces 
# initialize COCO API for instance annotations
dataDir = './'

image_folder = "extracted_generated_faces"

savedirimage = dataDir + "cocoapi/images/" + image_folder + "/"



if not os.path.exists(savedirimage):
	os.makedirs(savedirimage)

In [ ]:
# install all packages
from keras.models import load_model
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

import numpy as np
from mtcnn.mtcnn import MTCNN
from numpy import asarray


# Simple python package to shut up Tensorflow warnings and logs.
!pip install silence_tensorflow
import silence_tensorflow.auto

In [ ]:
# extract a single face from a given photograph
def extract_face(filename, index, required_size=(250, 250), threshold = 0.99):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    if len(results) >=1 and results[0]['confidence'] >= threshold:
        x1, y1, width, height = results[0]['box']
        if height-width>=15:
            # bug fix
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            # extract the face
            face = pixels[y1:y2, x1:x2]
            # resize pixels to the model size
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = np.asarray(image)
            plt.imsave(os.path.join(savedirimage, 'face_img_{}.jpg'.format(index)),face_array)
            return True
    return False